## Задание 1 (я писал прямо в тетрадке Ngrams с занятия)

## Языковое моделирование

Языковое моделирование заключается в приписывании вероятности последовательности слов. Сейчас языковые модели используются практически во всех nlp задачах. Всякие Берты и Элмо - языковые модели. 

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

Возьмем два разных корпуса: новостной и сообщения с 2ch.

In [1]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('data/2ch_corpus.txt', encoding='utf8').read()[:700000]
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные

news = open('data/lenta.txt', encoding='utf8').read()[:700000]

По длине оно сопоставимы.

In [2]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 700000
Длина 2 - 700000


Напишем простую функцию для нормализации. 

In [3]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [4]:
norm_dvach = normalize(dvach)[:500000]
norm_news = normalize(news)

In [5]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 110912
Длина корпуса новостных текстов в токенах -  90323


И по уникальным токенам

In [6]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 26166
Уникальный токенов в корпусе новостных текстов -  22569


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [7]:
from collections import Counter

In [8]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)


In [9]:
vocab_dvach.most_common(10)

[('и', 3144),
 ('в', 2921),
 ('не', 2736),
 ('на', 1927),
 ('что', 1623),
 ('а', 1266),
 ('я', 1223),
 ('с', 1171),
 ('это', 1078),
 ('как', 978)]

In [10]:
vocab_news.most_common(10)

[('в', 4264),
 ('и', 2145),
 ('на', 1643),
 ('по', 1214),
 ('что', 1042),
 ('с', 996),
 ('не', 813),
 ('как', 534),
 ('из', 482),
 ('о', 448)]

Сравнивать употребимость конкретных слов в разных текстах в абсолютных числах неудобно. Нормализуем счётчики на размеры текстов. Так у нас получается вероятность слова.

In [11]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(20)

[('и', 0.0283467974610502),
 ('в', 0.026336194460473167),
 ('не', 0.024668205424120022),
 ('на', 0.017374134448932488),
 ('что', 0.014633222735141373),
 ('а', 0.011414454702827467),
 ('я', 0.011026759953837277),
 ('с', 0.010557919792267743),
 ('это', 0.009719417195614541),
 ('как', 0.008817801500288517),
 ('ты', 0.008276832083092902),
 ('но', 0.006149019042123486),
 ('у', 0.006067873629544143),
 ('то', 0.005788372763993075),
 ('по', 0.0051121609924985574),
 ('так', 0.005031015579919215),
 ('все', 0.004868724754760531),
 ('если', 0.00482364396999423),
 ('же', 0.004183496826312753),
 ('для', 0.00409333525678015)]

In [12]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.047208352246935995),
 ('и', 0.023748104026659875),
 ('на', 0.018190272688019662),
 ('по', 0.013440651882687688),
 ('что', 0.011536375009687455),
 ('с', 0.01102709165993158),
 ('не', 0.00900102963807668),
 ('как', 0.005912115408035605),
 ('из', 0.0053364037952680935),
 ('о', 0.00495997697153549),
 ('к', 0.003952481649192343),
 ('россии', 0.003897124763349313),
 ('за', 0.003697839974314405),
 ('для', 0.0033103417734131947),
 ('он', 0.003144271115884105),
 ('от', 0.003055700098535257),
 ('его', 0.0029560577040178026),
 ('также', 0.002878558063837561),
 ('сша', 0.0025796308802851988),
 ('а', 0.0025574881259479865)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос - это предложение больше подходит для новостей или для анонимного форума?

In [13]:
phrase = 'Бои у Сопоцкина и Друскеник закончились отступлением германцев'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [14]:
prob.most_common()

[('dvach', 0.034414671090594345), ('news', 0.025430953356287988)]

In [15]:
phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [16]:
prob.most_common()

[('news', 0.05629795290236152), ('dvach', 0.04891265147143682)]

Результаты получаются не очень точные. Возможно это из-за того, что мы считаем слова независимыми друг от друга. А это очевидно не так

По-хорошему вероятность последовательности нужно расчитывать по формуле полной вероятности. Но у нас не очень большие тексты и мы не можем получить вероятности для длинных фраз (их просто может не быть в текстах). Поэтому мы воспользуемся предположением Маркова и будем учитывать только предыдущее слово.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма.

In [17]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [18]:
n = 3

Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [19]:
sentences_dvach = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [20]:
unigrams_dvach = Counter()
nminusonegrams_dvach = Counter()
ngrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    nminusonegrams_dvach.update(ngrammer(sentence, n=n-1)) #вот тут создаётся лишняя биграмма слово end, для которой нет триграммы
    ngrams_dvach.update(ngrammer(sentence))

unigrams_news = Counter()
nminusonegrams_news = Counter()
ngrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    nminusonegrams_news.update(ngrammer(sentence, n=n-1))
    ngrams_news.update(ngrammer(sentence))


In [21]:
len(unigrams_dvach)

26168

In [22]:
ngrams_news.most_common(10)

[('<start> <start> в', 408),
 ('<start> <start> по', 350),
 ('<start> <start> как', 250),
 ('<start> <start> на', 115),
 ('<start> <start> об', 90),
 ('<start> <start> однако', 86),
 ('<start> об этом', 86),
 ('<start> по словам', 82),
 ('<start> как сообщает', 77),
 ('<start> <start> он', 67)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первых n-1 слов

In [23]:
phrase =  'Бои у Сопоцкина и Друскеник закончились отступлением германцев'
# phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'
prob = Counter()
for ngram in ngrammer(['<start>', '<start>'] + normalize(phrase) + ['<end>']):
    words = ngram.split()
    
    word1 = ' '.join(words[:-1])
    
    if word1 in nminusonegrams_dvach and ngram in ngrams_dvach:
        prob['dvach'] += np.log(ngrams_dvach[ngram]/nminusonegrams_dvach[word1])
    else:
        prob['dvach'] += np.log(0.001)
    
    if word1 in nminusonegrams_news and ngram in ngrams_news:
        prob['news'] += np.log(ngrams_news[ngram]/nminusonegrams_news[word1])
    else:
        prob['news'] += np.log(0.001)

prob['news'] = np.exp(prob['news'])
prob['dvach'] = np.exp(prob['dvach'])

In [24]:
prob.most_common()

[('news', 0.00021920210434020194), ('dvach', 1.0000000000000022e-27)]

Работает получше. Мы воспользовались небольшим хаком - для слов или биграммов, которых не было у нас в словаре, прибавляли низкую вероятность. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов. Если интрересно - в книге Журафского про это есть.

Проблем с неизвестными словами у нас не будет, если мы будем пытаться сгенерировать новый текст. Давайте попробуем это сделать.

In [25]:
matrix_dvach = np.zeros((len(nminusonegrams_dvach), 
                   len(unigrams_dvach)))
id2gram_dvach = list(nminusonegrams_dvach)
gram2id_dvach = {gram:i for i, gram in enumerate(id2gram_dvach)}

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

for ngram in ngrams_dvach:
    words = ngram.split()
    word1 = ' '.join(words[:-1])
    last_word = words[-1]
    
    matrix_dvach[gram2id_dvach[word1]][word2id_dvach[last_word]] =  (ngrams_dvach[ngram]/
                                                                     nminusonegrams_dvach[word1])



In [26]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_news = np.zeros((len(nminusonegrams_news), 
                   len(unigrams_news)))
id2gram_news = list(nminusonegrams_news) 
gram2id_news = {gram:i for i, gram in enumerate(id2gram_news)}

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)} 


# вероятность расчитываем точно также
for ngram in ngrams_news:
    words = ngram.split()
    word1 = ' '.join(words[:-1])
    last_word = words[-1]
    
    matrix_news[gram2id_news[word1]][word2id_news[last_word]] =  (ngrams_news[ngram]/
                                                                     nminusonegrams_news[word1])



Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [27]:

def generate(matrix, id2word, gram2id, id2gram, n=100, start='<start> <start>'):
    text = []
    current_idx = gram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = '<start> <start>'
            current_x = chosen
        else:
            current_x = ' '.join(id2gram[current_idx].split()[1:]) + ' ' + id2word[chosen]
        current_idx =  gram2id[current_x]
    
    return ' '.join(text)

In [28]:
print(generate(matrix_dvach, id2word_dvach, gram2id_dvach, id2gram_dvach).replace('<end>', '\n'))

а как минимум неудобно 
 даже даун маск в секторе it смог в своё время ебало мамку омегосущества и твою скорее всего платят очень мало времени уделяешь языку 
 лейтенант отобрал твой значок потому что случай как говорит тяжелый и предлагает ехать или в оон опустят 
 а что такое издач с такой завязкой ничего нормального не получится 
 цыпа хороша да 
 уж очень не хочется же сливать в голивудщину когда после главы эдакое to be continued 
 если смотреть без перспективы и перпендикулярно т е мир на земле был бы изобрести ту вещь 
 хоронили арсекеда спалили два


In [29]:
print(generate(matrix_news, id2word_news, gram2id_news, id2gram_news).replace('<end>', '\n'))

повидимому под обломками также оказались десятки припаркованных неподалеку автомашин 
 такие учения проводятся не только в россии досрочных президентских выборов в йемене погибйеменец передало агентство риа новости впосольстве рф в дагестане лужков не исключил фактор мщения со стороны руководящего состава за организацией несения службы передает итар-тасс по словам рушайло взрывчатка обнаружена уже после утреннего взрыва на манежной площади в центре управления полетом в пасадене 
 как предполагает мк семья ельцина начинает понимать что немощный старик у руля государства опасен не только на 3 суток работы 
 путин высказался также за парковкой возле домов 
 это произойдет если будет хозяина склада


## Задание 2 (тут копировал куски из тетрадки про коллокации)

In [30]:
import itertools
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
from nltk.corpus import stopwords

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [31]:
corpus = open('data/lenta.txt', encoding='utf8').read()[:500000]

In [32]:
corpus = preprocess(corpus)

# Моя функция 

In [33]:
import math

In [34]:
def my_scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    try:
        if bigram_count - min_count > 0: #сразу выкинем низкочастотные биграммы
            score_ = (pow(bigram_count - min_count, 1.1) * len_vocab / ((worda_count * wordb_count))) #чуть-чуть увеличим вес биграммы
 #           score_ = math.log(score_, 2) с этой строкой (и return float('-inf') ниже) почему-то была ошибка с трешхолдом меньше -1
        else:
            return 0
    except ZeroDivisionError:
        return 0
    
    return score_

In [35]:
def collect_stats(corpus, stops):
    ## соберем статистики для отдельных слов
    ## и n-грамм
    
    unigrams = Counter()
    bigrams = Counter()
    
    for sent in corpus:
        unigrams.update(sent)
        bigrams.update(ngrammer(sent, stops, n=2))
    
    return unigrams, bigrams

In [36]:
def score_bigrams(unigrams, bigrams, scorer, threshold=float('-inf'), min_count=3):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigrams[bigram], len_vocab, min_count, None)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [37]:
unigrams, bigrams = collect_stats(corpus, stops)

In [38]:
bigram2score = score_bigrams(unigrams, bigrams, my_scorer)

In [39]:
bigram2score.most_common(20)

[(('дель', 'понтить'), 805.2877088112692),
 (('буэнос', 'айрес'), 805.2877088112692),
 (('луис', 'фри'), 805.2877088112692),
 (('usa', 'today'), 748.7592036090309),
 (('della', 'sera'), 731.2440615543088),
 (('corriere', 'della'), 700.7823230385126),
 (('геннадий', 'селезнёв'), 641.7936030934551),
 (('кубический', 'сантиметр'), 599.3652750073419),
 (('сан', 'франциско'), 587.0),
 (('петербургский', 'стиль'), 587.0),
 (('grand', 'сentral'), 587.0),
 (('борисовский', 'пруд'), 587.0),
 (('associated', 'press'), 576.1540215185414),
 (('холерный', 'вибрион'), 575.2055062937637),
 (('вынужденный', 'переселенец'), 575.2055062937637),
 (('каширский', 'шоссе'), 535.1723785175188),
 (('стихийный', 'бедствие'), 530.3856304558979),
 (('взрывчатый', 'вещество'), 506.24588876836765),
 (('исполнять', 'обязанность'), 471.45389373857586),
 (('шкала', 'рихтер'), 469.6)]

# Генсим

In [40]:
import gensim

In [41]:
# собираем статистики
ph = gensim.models.Phrases(corpus, min_count=3, threshold=-1, scoring=my_scorer)

In [42]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

In [43]:
p[corpus[157]][:15]

['штаб_поддержка',
 'владимир_семёнов',
 'отмечать_прокуратура',
 'министерство_внутренний',
 'дело_карачаево',
 'черкесия_практически',
 'принимать_мера',
 'пресечение_подобный',
 'действие_организовать',
 'лишь_оцепление',
 'омон_здание',
 'правительство_состояться',
 'сегодня_москва',
 'пресс_конференция',
 'бывший_шеф']

поднимем порог (тут я не видно попыток с другими порогами, потому что я запускал их в том же фрейме, а видно сразу результат)

In [44]:
ph = gensim.models.Phrases(corpus, min_count=3, threshold=125, scoring=my_scorer)
p = gensim.models.phrases.Phraser(ph)
p[corpus[157]][:15]

['штаб',
 'поддержка',
 'владимир_семёнов',
 'отмечать',
 'прокуратура',
 'министерство',
 'внутренний_дело',
 'карачаево_черкесия',
 'практически',
 'принимать',
 'мера',
 'пресечение',
 'подобный',
 'действие',
 'организовать']

Хотелось бы конечно штаб_поддержка и мера_пресечения, но они не проходят min_count 

In [45]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus], min_count=3, threshold=125, scoring=my_scorer)
p2 = gensim.models.phrases.Phraser(ph2)

In [46]:
p2[p[corpus[157]]][:15]

['штаб',
 'поддержка',
 'владимир_семёнов',
 'отмечать',
 'прокуратура',
 'министерство_внутренний_дело',
 'карачаево_черкесия',
 'практически',
 'принимать',
 'мера',
 'пресечение',
 'подобный',
 'действие',
 'организовать',
 'лишь']

In [ ]:
На этом куске получилось вполне нормально, министерство_внутренних_дел склеилось как надо. Попробуем на другом

In [49]:
p[corpus[57]][:15]

['заявить',
 'сегодня',
 'встреча',
 'премьер_министр',
 'япония',
 'вица_премьер',
 'правительство',
 'россия',
 'виктор',
 'христенко']

In [51]:
p2[p[corpus[57]]][:15]

['заявить',
 'сегодня',
 'встреча',
 'премьер_министр',
 'япония',
 'вица_премьер',
 'правительство',
 'россия',
 'виктор',
 'христенко']

Тут немного похуже(потерялся виктор_христенко и правительство_россии), но зато есть вице_премьер (который не склеился с японией) и премьер_министр